In [6]:
import sys
import os
sys.path.append(os.path.abspath("../"))  # or "../../" depending on location
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [7]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch.nn.functional as F
from utils.load_models import set_device, get_CNN_small, get_CNN, get_resnet
import numpy as np


# ==========================
# 変換定義（正規化なし）
# ==========================
transform = transforms.ToTensor()
normalize = transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                                 std=(0.2023, 0.1994, 0.2010))

# ==========================
# CIFAR-10データ読み込み
# ==========================

# ダウンロード済みのデータを指定して読み込み
root = "../data/"
train_dataset = datasets.CIFAR10(
    root=root, 
    train=True, 
    download=False,  
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
subset_size = 10000
indices = np.random.choice(len(train_dataset), subset_size, replace=False)

# サブセットデータセットを作成
small_train_dataset = Subset(train_dataset, indices)
train_loader_small = DataLoader(small_train_dataset, batch_size=128, shuffle=True)


# ==========================
# UAP生成関数（正規化前に摂動を加える）
# ==========================
def generate_uap(model, dataloader, xi=2/255, max_iter=3, p_norm='inf', device='cuda'):
    model.to(device)
    model.eval()
    delta = torch.zeros(1, 3, 32, 32).to(device)

    for itr in range(max_iter):
        print(f"[{itr+1}/{max_iter}] Updating UAP...")
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)

            # delta加算 → 正規化してモデルに渡す
            x_adv = torch.clamp(x + delta, 0, 1)
            x_adv = normalize(x_adv)

            with torch.no_grad():
                pred = model(x_adv).argmax(1)
            mask = pred == y
            if mask.sum() == 0:
                continue

            x_sel, y_sel = x[mask], y[mask]
            x_sel.requires_grad = True

            x_adv_sel = torch.clamp(x_sel + delta, 0, 1)
            x_adv_sel = normalize(x_adv_sel)
            out = model(x_adv_sel)
            loss = F.cross_entropy(out, y_sel)
            loss.backward()

            grad = x_sel.grad.data
            if p_norm == 'inf':
                v = xi * torch.sign(grad)
            elif p_norm == '2':
                v = xi * grad / (grad.norm(p=2, dim=(1,2,3), keepdim=True) + 1e-10)
            else:
                raise ValueError("p_norm must be 'inf' or '2'")

            delta = delta + v.mean(0, keepdim=True)
            delta = torch.clamp(delta, -xi, xi)

    return delta.detach()


In [8]:
# ==========================
# UAP生成と保存
# ==========================
device = set_device()
CNN_small = get_CNN_small().to(device)
uap_delta_sCNN = generate_uap(CNN_small, train_loader_small, xi=8/255, max_iter=3, p_norm='inf', device=device)
save_path = "uap/uap_delta_CNN_small.pt"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(uap_delta_sCNN, save_path)

[1/3] Updating UAP...
[2/3] Updating UAP...
[3/3] Updating UAP...


In [9]:
CNN = get_CNN().to(device)
uap_delta_CNN = generate_uap(CNN, train_loader, xi=8/255, max_iter=3, p_norm='inf', device=device)
save_path = "uap/uap_delta_CNN.pt"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(uap_delta_CNN, save_path)

[1/3] Updating UAP...
[2/3] Updating UAP...
[3/3] Updating UAP...


In [10]:
ResNet = get_resnet().to(device)
uap_delta_resnet = generate_uap(ResNet, train_loader, xi=8/255, max_iter=3, p_norm='inf', device=device)
save_path = "uap/uap_delta_resnet.pt"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(uap_delta_resnet, save_path)

[1/3] Updating UAP...
[2/3] Updating UAP...
[3/3] Updating UAP...
